## Empirical Bayes (EB) Method for the Unit Level Model

For complex (non-linear) parameters of interest or for non-normally distributed random errors, the EBLUP is not the best predictor and the MSE second-order approximation does not hold. Instead, we will use the Empirical Bayes (EB) method which fits the linear mixed model to the data and use a Monte Carlo (MC) approach to predict the area level complex parameters and their MSE estimates. Thsi approach is implemented by the *EBUnitLevel* class. 

To illustrate the EB method, we will simulate a data following the specification of Molina and Rao (2010). In their paper, they simulated data to analyze the performance of the EB method to estimate area poverty incidences and poverty gaps. 

### Simulated census data

The simulated census data is composed of $N=20,000$ observations from $m=80$ areas with $N_i = 250$ units in each area $i = 1, ..., m,$ with $k_{ij} = 1$. The population are generated using the linear mixed model with random area effects variance $\sigma_u^2 = 0.15^20$ and unit level errors variance $\sigma_e^2 = 0.5^2$. We consider two binary auxiliary variables simulated with $p_{1i} = 0.3+0.5*i/m$ and $p_{2i} = 0.2,$ with $i = 1, ..., m$. The output variable is income. Income is a skewed distribution and to simulated the skewness, we apply an exponential transformation to Y obtained from the linear mixed model discribed above.

In [7]:
%load_ext lab_black
import numpy as np
import pandas as pd

import samplics
from samplics.sampling import Sample
from samplics.sae import EbUnitModel

In [8]:
np.random.seed(123)

# model parameters
scale = 1
sigma2e = 0.5 ** 2
sigma2u = 0.15 ** 2

# Population sizes
N = 20000
nb_areas = 80

# Errors generation
error = np.random.normal(loc=0, scale=(scale ** 2) * (sigma2e ** 0.5), size=N)
area = np.sort(np.random.choice(range(1, nb_areas + 1), N))
areas, Nd = np.unique(area, return_counts=True)
random_effects = np.random.normal(loc=0, scale=sigma2u ** (1 / 2), size=nb_areas)
total_error = np.repeat(random_effects, Nd) + error

# Auxiliary information
p1 = 0.3 + 0.5 * np.linspace(1, nb_areas + 1, nb_areas) / nb_areas
p2 = 0.2
X1 = np.array([]).astype(int)
for i, d in enumerate(areas):
    Xk = np.random.binomial(1, p=p1[i], size=Nd[i])
    X1 = np.append(X1, Xk)
X2 = np.random.binomial(1, p=p2, size=N)
X = np.column_stack((np.ones(N), X1, X2))

beta = np.array([3, 0.03, -0.04])
Y = np.matmul(X, beta) + total_error
income = np.exp(Y)

# Create a dataframe for the population data
census_data = pd.DataFrame(data={"area": area, "X1": X1, "X2": X2, "income": income})

nb_obs = 15
print(f"\nFirst {nb_obs} rows of the population data\n")
census_data.head(nb_obs)


First 15 rows of the population data



,area,X1,X2,income
0,1,1,0,10.910632
1,1,0,0,30.000852
2,1,0,0,20.989920
3,1,0,0,8.579758
4,1,0,1,13.108396
5,1,0,0,41.607121
6,1,0,1,5.202886
7,1,0,0,14.703665
8,1,1,0,35.357877
9,1,0,0,11.812790


Now we re going to select a sample of 50 units in each area. To select the sample, we will use the *Sample* class from *samplics*. 

In [9]:
sample_size = 50
stratum = census_data["area"]
unit_id = census_data.index

sae_sample = Sample(method="srs", stratification=True, with_replacement=False)
sample, _, _ = sae_sample.select(unit_id, sample_size, stratum)
sample_data = census_data[sample == 1]

nb_obs = 15
print(f"\nFirst {nb_obs} rows of the sample data\n")
sample_data.head(nb_obs)


First 15 rows of the sample data



,area,X1,X2,income
0,1,1,0,10.910632
6,1,0,1,5.202886
11,1,1,0,17.907123
13,1,0,0,13.238140
18,1,0,0,30.101651
19,1,0,1,21.234904
24,1,1,0,10.030322
36,1,1,0,18.802240
43,1,0,0,24.275218
45,1,0,0,18.113143


Now that we have simulated a population(census) and selected a sample from it, we are ready to illustrate the steps of the EB method. In summary, we need the sample data with the output variable, the auxiliary variables, and the area variable. And we also need the non-sampled values of the auxiliary variables. In practice, it may challenging to get the out-of-sample auxiliary variables and even more difficult to link the sample to the census in order to partition it into sample vs out-of-sample.

In [10]:
# Sample data
areas = sample_data["area"]
income = sample_data["income"]
Xs = sample_data[["X1", "X2"]]

# Out of sample data
outofsample_data = census_data[sample != 1]
arear = outofsample_data["area"]
Xr = outofsample_data[["X1", "X2"]]

### Fitting the linear mixed model



In [11]:
eb_poverty_reml = EbUnitModel(method="REML", boxcox=0)
eb_poverty_reml.fit(income, Xs, areas, intercept=True)

print(f"\nThe estimated fixed effect using REML is: {eb_poverty_reml.fixed_effects}")
print(
    f"\nThe estimated area random effect standard error using REML is: {eb_poverty_reml.re_std}"
)
print(
    f"\nThe estimated area random effect standard error using REML is: {eb_poverty_reml.error_std}"
)


The estimated fixed effect using REML is: [ 2.97923793  0.03629499 -0.02642623]

The estimated area random effect standard error using REML is: 0.15227754550016784

The estimated area random effect standard error using REML is: 0.4972377422416066


In [12]:
eb_poverty_ml = EbUnitModel(method="ML", boxcox=0)
eb_poverty_ml.fit(income, Xs, areas, intercept=True)

print(f"\nThe ML estimated fixed effect using ML is: {eb_poverty_ml.fixed_effects}")
print(
    f"\nThe ML estimated area random effect standard error using ML is: {eb_poverty_ml.re_std}"
)
print(
    f"\nThe ML estimated area random effect standard error using ML is: {eb_poverty_ml.error_std}"
)


The ML estimated fixed effect using ML is: [ 2.97921929  0.03631835 -0.02639921]

The ML estimated area random effect standard error using ML is: 0.15109887793783536

The ML estimated area random effect standard error using ML is: 0.4971123962940577


### Prediction of area level poverty indicators

In [10]:
poverty_incidence(y, poverty_line = 12)

poverty_gap(y, poverty_line = 12)

NameError: name 'poverty_incidence' is not defined

In [16]:
eb_poverty_reml.predict(Xr, arear, intercept = True)

TypeError: predict() missing 2 required positional arguments: 'indicator' and 'number_samples'